In [17]:
!pip install stable-baselines3 sb3-contrib --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.2/93.2 kB 3.2 MB/s eta 0:00:00


In [12]:
from google.colab import drive
drive.mount('/content/drive')

!cp -r /content/drive/MyDrive/HFT-RL/src/envs  /content/

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import polars as pl
import pandas as pd

import stable_baselines3
from stable_baselines3.common.vec_env import DummyVecEnv

import numpy as np
import matplotlib.pyplot as plt

import gymnasium as gym
from envs.env import TradingEnv
import pandas as pd

### *Loading Data*

*Based on NASDAQ Technical 100 Stock LOB*  
*Duration - 2025-05-04 to 2025-06-04*  
*Time Interval - 1 minute*

In [8]:
df = pl.read_csv("/content/drive/MyDrive/HFT-RL/data/USATECHIDXUSD_mt5_ticks_2.csv")

# Convert Timestamp column to datetime
df_with_datetime = df.with_columns(
    pl.col("Timestamp").str.strptime(pl.Datetime, format=
    "%Y%m%d %H:%M:%S:%3f").alias("datetime")
)

df = df_with_datetime.drop("Timestamp")
df = df.to_pandas()

In [9]:
df[['Bid price', 'Bid volume']].head(10)

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


,Bid price,Bid volume
0,20046.264,0.00009
1,20046.109,0.00009
2,20046.121,0.00009
3,20046.109,0.00009
4,20046.154,0.00009
5,20046.232,0.00009
6,20046.787,0.00009
7,20046.387,0.00009
8,20046.565,0.00009
9,20046.832,0.00009


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [ ]:
env = TradingEnv(df)
state = env.reset()
portfolio_value = env.initial_balance
tracker = [portfolio_value]
while True:
    action = env.action_space.sample()  # Random action
    n_state, reward, done, _, info = env.step(action)
    portfolio_value += reward
    tracker.append(portfolio_value)
    if done:
        print("info", info)
        break
plt.figure(figsize = (15,6))
plt.plot(tracker, label='Portfolio Value')
# env.render_all()
plt.show

info {'cash_in_hand': np.float32(9971.522), 'shares_held': np.float32(0.0014129956), 'net_worth': np.float32(9999.851)}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


NameError: name 'tracker' is not defined

<Figure size 1500x600 with 0 Axes>

In [20]:
from stable_baselines3 import A2C
from stable_baselines3.common.vec_env import DummyVecEnv
from sb3_contrib import RecurrentPPO

vec_env = DummyVecEnv([lambda: TradingEnv(df, episode_length=100000)])
policy_kwargs = dict(
    net_arch=[64, 64],
    activation_fn="relu",
)
model = RecurrentPPO("MlpLstmPolicy", vec_env, verbose=1)
model.learn(total_timesteps=100000)


Using cpu device


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Streaming output truncated to the last 5000 lines.
|    time_elapsed         | 493       |
|    total_timesteps      | 64640     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0.2       |
|    entropy_loss         | -0.000715 |
|    explained_variance   | -1.19e-07 |
|    learning_rate        | 0.0003    |
|    loss                 | 7.08e-08  |
|    n_updates            | 5040      |
|    policy_gradient_loss | -1.05e-07 |
|    value_loss           | 1.35e-08  |
---------------------------------------
---------------------------------------
| time/                   |           |
|    fps                  | 130       |
|    iterations           | 506       |
|    time_elapsed         | 494       |
|    total_timesteps      | 64768     |
| train/                  |           |
|    approx_kl            | 0.0       |
|    clip_fraction        | 0         |
|    clip_range           | 0

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [21]:
# Evaluate the trained QRDQN model on the TradingEnv and plot portfolio value over an episode

env = TradingEnv(df)
state = env.reset()
portfolio_values = []

while True:
    # Use only the observation part for model prediction
    obs = state[0] if isinstance(state, tuple) else state
    action = model.predict(obs, deterministic=True)
    n_state, reward, done, _, info = env.step(action)
    # portfolio_values.append(tracker)
    state = n_state
    if done:
        print("info", info)
        break

# plt.figure(figsize=(15, 6))
# plt.plot(portfolio_values, label='Portfolio Value')
# plt.xlabel('Step')
# plt.ylabel('Portfolio Value')
# plt.title('Portfolio Value Over Episode (QRDQN Policy)')
# plt.legend()
# plt.show()

info {'cash_in_hand': 10000, 'shares_held': 0.0, 'net_worth': np.float32(10000.0)}


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
